In [38]:
import numpy as np
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
from collections import defaultdict
from os import listdir
from os.path import isfile, join

In [39]:
# Hyperparameters
max_per_slot = 2 # Max number of candidate responses to extract (per iteration)
max_span_length = 20 # Max length of one answer span
min_score = 0 # Min score for an answer span

In [40]:
class QA_Model:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased-distilled-squad")
        self.model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-cased-distilled-squad")
    
    def answer(self, text, questions):
        answerss = []
        for question in questions:
            inputs = self.tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt")
            input_ids = inputs["input_ids"].tolist()[0]

            text_tokens = self.tokenizer.convert_ids_to_tokens(input_ids)
            answer_start_scores, answer_end_scores = self.model(**inputs, return_dict=False)

            answer_start = torch.argmax(answer_start_scores)  # Get the most likely beginning of answer with the argmax of the score
            answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score
            score = torch.max(answer_start_scores) + torch.max(answer_end_scores)
            
            if answer_start < 1: 
                answer = "[None]" # cannot start with CLS
            elif answer_end - answer_start + 1 > max_span_length:
                answer = "[None]" # cannot be longer than hyperparam
            elif score < min_score:
                answer = "[None]" # cannot be < hyperparam
            else:
                answer = self.tokenizer.convert_tokens_to_string(self.tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
#                 print(f"Question: {question}")
#                 print(f"Answer: {answer}")
#                 print(f"Score: {score}") 
                
            answerss.append(answer)
        return answerss

In [18]:
folder = '../data/'
file = 'event_1054347.txt'

questions = ["Where did the theft take place?",
             "What was stolen?" ,
             "When was the item last seen?",
             "When was the item stolen?"]

qa = QA_Model()

In [21]:
answers = []
for q in questions:
    answers.append([""])
print(answers)
f = open(folder + file)
text = f.readlines()

for i in range(len(text)):
    print(str(i) + " " + text[i])
    
for i in range(len(text)):
    print(f"Up to line {i}")
    section = " ".join(text[:i+1])
    answers_i = qa.answer(section, questions)
    for j in range(len(answers_i)):
        if answers_i[j] not in answers[j]:
            answers[j].append(answers_i[j])

# display by printing
print("Final Answers ==================")
for i in range(len(questions)):
    print(questions[i])
    print(answers[i])

[[''], [''], [''], [''], ['']]
0 User "This evening ([DATE]), between #:## [ACRONYM] and #:## [ACRONYM] my bike lights (both front and rear) were stolen off my bike in front of [ORG]. The bike was parked at a sign directly in front of a security camera, and I was wondering if I could view who stole my lights. I also know that a security guard constantly stands in front of the academic building, checking student IDs. Would I be able to contact that security guard regarding my stolen lights? Thank you for your understanding."

1 Admin "Thank you for contacting [ORG]. We can reach out to the officer to see if anything suspicious happened around the bike rack during their shift."

2 Admin "Would you like to file a report at this time?"

3 User "Yes, please. My bike wasn't on the rack around the building; it was near the front of the entrance, locked to a sign. Thank you so much."

4 Admin "Are you currently at [FAC]"

5 User "No, I have returned home."

6 Admin "[ACRONYM], Reports are typi

In [41]:
class DialogueStateTracking:
    def __init__(self):
        qa = QA_Model()
        slot_temp = [] # slot template
        slot_questions = [] # questions corresponding to slot template
        
    # dialogue: list of strings (each utterance in a new string)
    # returns: answers: list of set of tuples. for each utterance, the slots predicted for the dialogue history
    def predict_slots(self, dialogue):
        answers = [] # length of dialogue
        for i in range(len(dialogue)):
            section = " ".join(dialogue[:i+1])
            answers_i = qa.answer(section, self.slot_questions)
            answers_i_dict = {}
            for j in range(len(answers_i)):
                # TODO: filter based on score, so you don't fill every slot every turn
                answers_i_dict[self.slot_temp[j]] = answers_i[j]
            answers.append(answers_i_dict)
        return answers

In [42]:
# script to run slot filling on ../data/ output answers and eval
dst = DialogueStateTracking()
dst.slot_temp = ["Location", "Object_stolen", "Time_start", "Time_end"]
dst.slot_questions = ["Where did the theft take place?",
                     "What was stolen?" ,
                     "When was the item last seen?",
                     "When was the item stolen?"]

val_data_path = "../data/val/"
fnames = [f for f in listdir(val_data_path) if isfile(join(val_data_path, f))]
for fname in fnames:
    # TODO: get root of name
    f = open(val_data_path + fname)
    dialogue = f.readlines()
    slots_pred = dst.predict_slots(dialogue)
    print(slots_pred)
    f.close()


Question: Where did the theft take place?
Answer: road sign
Score: 2.2792413234710693
Question: What was stolen?
Answer: a [ ORG ] road sign
Score: 21.002765655517578
Question: When was the item stolen?
Answer: a [ ORG ] road sign
Score: 0.7307089567184448
Question: Where did the theft take place?
Answer: road sign
Score: 4.138988018035889
Question: What was stolen?
Answer: a [ ORG ] road sign
Score: 20.452049255371094
Question: When was the item stolen?
Answer: a [ ORG ] road sign
Score: 2.320535659790039
Question: Where did the theft take place?
Answer: lakeside dorms
Score: 11.4732027053833
Question: What was stolen?
Answer: a [ ORG ] road sign
Score: 20.149078369140625
Question: When was the item stolen?
Answer: a [ ORG ] road sign
Score: 0.16790443658828735
Question: Where did the theft take place?
Answer: lakeside dorms
Score: 12.247528076171875
Question: What was stolen?
Answer: a [ ORG ] road sign
Score: 20.24231719970703
Question: When was the item stolen?
Answer: a [ ORG ] ro

Question: What was stolen?
Answer: bicycle
Score: 17.13591194152832
Question: When was the item stolen?
Answer: 
Score: 3.8426170349121094
Question: Where did the theft take place?
Answer: outside of food city
Score: 12.75452995300293
Question: What was stolen?
Answer: bicycle
Score: 15.503941535949707
Question: When was the item stolen?
Answer: 
Score: 2.460709571838379
Question: Where did the theft take place?
Answer: outside of food city
Score: 13.007352828979492
Question: What was stolen?
Answer: bicycle
Score: 15.66446304321289
Question: When was the item stolen?
Answer: 
Score: 2.435135841369629
[{'Location': 'outside of food city', 'Object_stolen': 'bicycle', 'Time_start': 'bicycle', 'Time_end': ''}, {'Location': 'outside of food city', 'Object_stolen': 'bicycle', 'Time_start': '[None]', 'Time_end': ''}, {'Location': 'outside of food city', 'Object_stolen': 'bicycle', 'Time_start': '[None]', 'Time_end': ''}, {'Location': 'outside of food city', 'Object_stolen': 'bicycle', 'Time_